In [ ]:
import wandb
from wandb.keras import WandbCallback
import numpy as np
import random

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dropout, Input, Flatten, Dense, Concatenate
from tensorflow.keras.losses import CategoricalCrossentropy, Huber, MeanSquaredError
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

from tf_agents.environments import py_environment
from tf_agents.trajectories import trajectory
from tf_agents.trajectories import time_step as ts
from tf_agents.specs import array_spec, tensor_spec

from spektral.layers import XENetDenseConv
from spektral.utils.sparse import sp_matrix_to_sp_tensor
from spektral.data import Graph
from spektral.data.dataset import Dataset
from spektral.data.loaders import BatchLoader

In [ ]:
# initialize wandb

wandb.init(project="test-project", 
           entity="locp")

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: filippo_festa (locp). Use `wandb login --relogin` to force relogin
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


Parameters to be set:
- number of train_episodes
- size of replay_memory
- update steps of main and target network
- discount_factor
- MIN_REPLAY_SIZE (minimum size of the replay_memory for which we do the training)
- batch_size (size of the training dataset)

## Predefined Functions 

### Adjacency Matrix 

In [ ]:
def Adj(D, L, sparse=False):
    N = L**D

    # create all nodes' coordinates
    nodes = [x for x in np.ndindex(tuple(np.repeat(L,D)))]

    # Pass from coordinate to node's index
    # (h,...k,j,i) <=> index = h*L^(D-1) + ... + k*L^2 + j*L + i
    mul = [L**i for i in reversed(range(D))]

    # Creation of adjacency matrix 
    A_dense = []
    # creation of a row for each node's coordinate 
    for node in nodes:       
        temp_buffer = []
        A_dense_row = [0]*N
        # find the two nearest neighbours of the node along each dimension
        for d in range(D):
            temp=list(node)
            temp[d]=((temp[d]+1)%L)
            temp=np.inner(temp, mul)
            temp_buffer.append(temp)    

            temp=list(node)
            temp[d]=((temp[d]-1)%L)
            temp=np.inner(temp, mul)
            temp_buffer.append(temp)
      
        temp_buffer=list(np.unique(np.array(temp_buffer), axis=0))   
        for i in temp_buffer: A_dense_row[i]=1
        A_dense.append(A_dense_row)
    
    # sparse=False => sparse adjacency matrix
    # sparse=True => dense adjacency matrix
    if sparse:
        return sp_matrix_to_sp_tensor(np.array(A_dense))
    else:
        return np.array(A_dense)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Interaction Matrix 

In [ ]:
def J_inter(denseAdj):
    N = denseAdj.shape[0]
    sparseAdj = sp_matrix_to_sp_tensor(denseAdj)

    # sparse adjacency matrix as a numpy array
    edge=sparseAdj.indices.numpy()

    # ordered numpy sparse adjacency matrix
    un_edge=np.array([np.sort(i) for i in edge]) 

    # creation of the interaction array: (i,j) and (j,i) have the same Jij
    inter=[]
    for i in range(len(un_edge)):
        equal=True
        for j in range(i):
            if np.array_equal(un_edge[i],un_edge[j]):
                inter.append(inter[j])
                equal=False
                break
        if equal: 
            inter.append(np.random.normal(0, 1))
    
    # creation of dense interaction matrix
    inter_matrix = np.zeros((N,N))
    counter = 0
    for i, j in edge:
        inter_matrix[i,j] = inter[counter]
        counter += 1
    return [np.array(inter).reshape(sparseAdj.indices.shape[0],1), inter_matrix.reshape((N,N,1))]
    
    # index of the returned list:
    # 0 => interaction array
    # 1 => interaction matrix (zero padded)

### Dataset

In [ ]:
class MyDataset(Dataset):
    def __init__(self, N_graph, X, Y, A, E, **kwargs):
        self.X = X
        self.Y = Y
        self.N_graph = N_graph
        self.A = A
        self.E = E
        super().__init__(**kwargs)

    def read(self):
        mydataset = []
        for i in range(self.N_graph):
            # list of Graph objects that will be used as input in the BatchLoader
            mydataset.append(
                    Graph(x=self.X[i], a=self.A[i], e=self.E[i], y=self.Y[i])      
                    )
        return mydataset

### Replay memory buffer

In [ ]:
# save n-step transitions (s_t; a_t; r_t,t+n; s_t+n) from the trajectory buffer

def get_replay_memory(trajectory_buffer, replay_memory):
    n = len(trajectory_buffer)

    states = np.array([transition[0] for transition in trajectory_buffer])
    actions = np.array([transition[1] for transition in trajectory_buffer])
    rewards = np.array([transition[2] for transition in trajectory_buffer])
    done = np.array([transition[4] for transition in trajectory_buffer])
    inter_matrix = trajectory_buffer[0][-1]
    dense_AdjMat = trajectory_buffer[0][-2]

    cum_reward = np.cumsum(rewards)

    # creating the replay memory buffer from the trajectory one
    # => (starting state, action performed, cumulative reward after n step from the starting one, state after n step from the starting one, episode ended, dense adjacency matrix of the episode, interaction matrix of the episode)
    replay_memory.append([states[0], actions[0], cum_reward[n-1], states[n-1], done[n-1], dense_AdjMat, inter_matrix])

    return replay_memory

## Environment 

In [ ]:
class SG_env(py_environment.PyEnvironment):

  def __init__(self, L, D):
    # Initialize environment attributes
    # - action_spec: action declaration (integer from 0 to N-1)
    # - observation_spec: state of the system declaration
    # - episode_ended: flag for the end of the episode (all spin down)
    # - state: state of the system (1 and -1 array)
    self.N = L**D
    self._action_spec = array_spec.BoundedArraySpec(
        shape=(), dtype=np.int32, minimum=0, maximum=self.N-1, name='action')
    self._observation_spec = array_spec.BoundedArraySpec(
        shape=(self.N,1), dtype=np.int32, minimum=-1, maximum=1, name='observation')
    self.sp_AdjMat = Adj(D, L, sparse=True)
    self.dense_AdjMat = Adj(D, L, sparse=False)
    self.interaction = J_inter(self.dense_AdjMat)[0]   
    self.inter_matrix = J_inter(self.dense_AdjMat)[1]
    self._state = np.ones(shape=(self.N,1)).astype("int32")
    self._episode_ended = False

  def get_state(self):
    return self._state

  def show_N(self):
    return self.N  

  def action_spec(self):
    return self._action_spec

  def observation_spec(self):
    return self._observation_spec

  def show_dense_AdjMat(self):
    return self.dense_AdjMat

  def show_sp_AdjMat(self):
    return self.sp_AdjMat

  def show_interaction(self):
    return self.interaction

  def show_inter_matrix(self):
    return self.inter_matrix

  # True => All spins = -1, False => otherwise
  def __all_spins_down(self):
    return np.all(self._state==-1)    

  # Compute the reward of the chosen action 
  # reward = energy difference between consecutive states
  # nns => nearest neighbours indexes
  # nn_Js => nearest neighbours interactions' indexes
  def computeReward(self, action):
    nns = self.sp_AdjMat.indices[self.sp_AdjMat.indices[:,0]==action][:,1].numpy()
    nn_Js = np.where(self.sp_AdjMat.indices[:,0]==action)[0]
    nn_sum = 0
    for i in range(len(nns)): nn_sum += self.interaction[nn_Js[i]]*self._state[nns[i],0]
    reward = 2*nn_sum*self._state[action,0]
    return reward[0]

  # Compute the energy of the current state 
  def computeEnergy(self):
    edge = self.sp_AdjMat.indices.numpy()
    Nedge = len(edge)
    energy = 0
    for i in range(Nedge):
        energy -= self.interaction[i][0]*self._state[edge[i][0]][0]*self._state[edge[i][1]][0]
    return energy/2

  # reset function: called when all spins are -1 => new episode
  #                                              => all spins up (=1) and new interaction matrix (if needed)
  def _reset(self):
    self._state = np.ones(shape=(self.N,1)).astype("int32")
    #self.interaction = J_inter(self.dense_AdjMat)[0]    
    #self.inter_matrix = J_inter(self.dense_AdjMat)[1]
    self._episode_ended = False
    return ts.restart(np.array(self._state, dtype=np.int32))

  # step function: describe the process of applying the action selected by the agent
  # ts.restart, ts.transition and ts.termination return a timestep 
  # containing step_type, reward, discount and observation
  def _step(self, action):
    if self._episode_ended:
      return self.reset()

    if self.__all_spins_down():
      self._episode_ended = True
    elif (action>=0 and action<=self.N-1) and (self._state[action,0]==1):
      self._state[action,0]=-1
      rew = self.computeReward(action)
      
      if self.__all_spins_down():
          self._episode_ended = True
          return ts.termination(np.array(self._state, dtype=np.int32), reward=rew)
      else:
          return ts.transition(np.array(self._state, dtype=np.int32), reward=rew)
    
    elif (action>=0 and action<=self.N-1) and (self._state[action,0]==-1):
      raise ValueError('Each spin can be flipped only once!')
    else:
      raise ValueError('`action` should be 0 up to N-1 - Spin Flip!')

## Agent

In [ ]:
# Agent (=> GNN+FNN) 
# N => number nodes
# D => number dimensions
# stack_channels => integer or list of integers, number of channels for the hidden layers
# node_channels => integer, number of output channels for the nodes
# edge_channels => integer, number of output channels for the edges
# division_factor_dense => integer, gradually reduce the number of neurons for each dense layer
# p_drop => float between 0 and 1, fraction of the input units to drop
# N_xenet => Number of XENetDenseConv layers
# N_dense => Number of Dense hidden layers

def agent(N,                                 
          D,                                 
          stack_channels=5,        
          node_channels=3,
          edge_channels=3,
          division_factor_dense=4,
          p_drop=0,
          N_xenet=2,
          N_dense=2,
          activation="relu",
          regularizer=0):
  inX = Input(shape=(N,1), name='Input Nodes')
  inA = Input(shape=(N,N), name='Input Adj matrix')
  inE = Input(shape=(N,N,1), name='Input Edges')
  
  XENet_layer = XENetDenseConv(stack_channels, node_channels, edge_channels,
                     attention=True, node_activation=activation, edge_activation=activation, kernel_regularizer=l2(regularizer), name="XENet_layer")
  X, E = XENet_layer([inX, inA, inE])
  for i in range(N_xenet-1):
    X, E = XENet_layer([X, inA, E])
  
  # flat the updated X, E in order to feed the fully connected neural network (FNN)
  flat_x, flat_e = Flatten(name="Nodes_encoding")(X), Flatten(name="Edges_encoding")(E)
  out = Concatenate(axis=1, name="Concatenation")([flat_x])    #,flat_e
  
  for i in range(N_dense):
    out = Dense(out.shape.as_list()[1]//division_factor_dense, activation=activation, kernel_regularizer=l2(regularizer))(out)
    out = Dropout(p_drop)(out)
  out = Dense(N, activation=activation, kernel_regularizer=l2(regularizer), name='Q-values')(out)
  
  model = Model([inX,inA,inE], out)
  model.compile(optimizer=Adam(), loss=MeanSquaredError())
  return model

## Training

In [ ]:
def train(env, replay_memory, model, target_model, done):
    discount_factor = 0.618
    
    # skip the training if the number of samples in the replay 
    # memory is less than MIN_REPLAY_SIZE
    MIN_REPLAY_SIZE = 32
    if len(replay_memory) < MIN_REPLAY_SIZE:
        return

    # randomly select a number of samples from the replay memory equal to batch_size
    batch_size = 32
    mini_batch = random.sample(replay_memory, batch_size)

    # sets of all interaction matrix and dense adjacency matrix in the batch
    E = np.array([transition[-1] for transition in mini_batch])
    A = np.array([transition[-2] for transition in mini_batch])

    # current_states => set of all starting observations in the batch
    # current_qs_list => predicted Q-values of the current_states by the model
    current_states = np.array([transition[0] for transition in mini_batch])
    current_qs_list = np.array(model.predict([current_states,A,E]))
 
    # new_current_states => set of observations after performing n actions in the batch
    # future_qs_list => predicted Q-values of the new_current_states by the target model
    new_current_states = np.array([transition[3] for transition in mini_batch])
    future_qs_list = np.array(target_model.predict([new_current_states,A,E]))
    
    # X => observations
    # Y => 'label' of each observation: updated current_qs_list 
    X = []
    Y = []
    for index, (observation, action, reward, new_observation, done, dense_AdjMat, inter_matrix) in enumerate(mini_batch):
        if not done:
            new_q = reward + discount_factor*np.max(future_qs_list[index])
        else:
            new_q = reward

        # update the Q-value corrisponding to the performed action
        current_qs = current_qs_list[index]
        current_qs[action] = new_q
        
        X.append(observation)
        Y.append(current_qs)

    # build the training dataset 
    train_data = MyDataset(N_graph=batch_size, X=X, Y=Y, A=A, E=E)
    # use the BatchLoader to fit the model and WandbCallback to load the loss to Weight&Biases
    loader = BatchLoader(train_data, node_level=False, epochs=50, batch_size=batch_size, shuffle=False) 
    model.fit(loader.load(), steps_per_epoch=loader.steps_per_epoch, verbose=2,
                callbacks = [WandbCallback()])


In [ ]:
def main():
    L = 3
    D = 2
    env = SG_env(L=L, D=D)

    # Epsilon-greedy algorithm in initialized at 1 
    # => every step is random at the start
    epsilon = 1          
    max_epsilon = 1     
    min_epsilon = 0.01  
    decay = 0.01

    # 1. Initialize the Target and Main models 
    # Main Model (updated every 3 steps)
    model = agent(N=env.N, D=D, N_xenet=1, N_dense=0, division_factor_dense=2)

    # Target Model (updated at the end of every episode)
    target_model = agent(N=env.N, D=D, N_xenet=1, N_dense=0, division_factor_dense=2)
    target_model.set_weights(model.get_weights())

    energy_buffer = []
    replay_memory = []
    
    train_episodes = 300
    for episode in range(train_episodes):
        # reset the variables at the beginning of an episode
        trajectory_buffer = []                   
        steps_to_update_target_model = 0
        env.reset()
        previous_obs = np.ones(shape=(env.N,1)).astype("int32")
        done = False  
        check = np.arange(0,env.N)

        while not done: 
            observation = env.get_state()          
            print("\n\t\t\t\t++++++++++++  episode:", episode," - step:", steps_to_update_target_model, " ++++++++++++")

            # 2. Explore using the Epsilon Greedy Exploration Strategy
            random_number = np.random.rand()
            if random_number <= epsilon:
                # Explore
                action = random.choice(check)

            else:
                # Exploit best known action
                predicted = model([observation.reshape(1,env.N,1), env.dense_AdjMat.reshape(1,env.N,env.N), env.inter_matrix.reshape(1,env.N,env.N,1)], training=False).numpy()[0]
                while True:
                    # check to prevent flipping the same spin twice - only once!
                    action = np.argmax(predicted)
                    if env.get_state()[action,0] == 1:
                            break;
                    predicted[action] = np.NINF

            # remove the choosen action from check array
            check = np.setdiff1d(check, action)

            # perform the action on the environment and get the updated parameters
            step_type, reward, discount, new_observation = env._step(action)
            done = env._episode_ended
            e = env.computeEnergy()

            # save the parameter in the buffer
            trajectory_buffer.append([previous_obs, action, reward, new_observation, done, e, env.dense_AdjMat, env.inter_matrix])
            energy_buffer.append([episode, new_observation, env.interaction, e])  
            
            # load interesting parameters to weight&Biases
            wandb.log({
                "Episode": energy_buffer[episode*env.N+steps_to_update_target_model][0],
                "Step": episode*env.N+steps_to_update_target_model,
                "New observation": energy_buffer[episode*env.N+steps_to_update_target_model][1],
                "J interactions": energy_buffer[episode*env.N+steps_to_update_target_model][2],
                "Energy": energy_buffer[episode*env.N+steps_to_update_target_model][3]
            })

            # fill the replay memory buffer
            if steps_to_update_target_model >= L:   
                replay_memory = get_replay_memory(trajectory_buffer, replay_memory)
                trajectory_buffer = trajectory_buffer[1:]

            # 3. Update the Main Network using the Bellman Equation  
            if (steps_to_update_target_model%L==0 and steps_to_update_target_model!=0) or done:
                print("\n\t\t\t\t\t      +++++ Training +++++")
                train(env, replay_memory, model, target_model, done)
  
            previous_obs = new_observation

            # Copying main network weights to the target network
            # weights at the end of the episode
            if done:
                if episode >= 5:
                    target_model.set_weights(model.get_weights())
                break

            steps_to_update_target_model += 1

        # update epsilon using the following rule
        epsilon = min_epsilon+ (max_epsilon -min_epsilon) * np.exp(-decay *episode)

In [ ]:
main()

				++++++++++++  episode: 277  - step: 7  ++++++++++++

					      +++++ Training +++++
1/1 - 0s - loss: 3.9174 - _timestamp: 1653930841.0000 - _runtime: 369.0000 - 12ms/epoch - 12ms/step

				++++++++++++  episode: 277  - step: 8  ++++++++++++

					      +++++ Training +++++
1/1 - 0s - loss: 3.8080 - _timestamp: 1653930841.0000 - _runtime: 369.0000 - 12ms/epoch - 12ms/step

				++++++++++++  episode: 278  - step: 0  ++++++++++++

					      +++++ Training +++++
1/1 - 0s - loss: 2.2597 - _timestamp: 1653930842.0000 - _runtime: 370.0000 - 16ms/epoch - 16ms/step

				++++++++++++  episode: 278  - step: 1  ++++++++++++

					      +++++ Training +++++
1/1 - 0s - loss: 3.7336 - _timestamp: 1653930842.0000 - _runtime: 370.0000 - 16ms/epoch - 16ms/step

				++++++++++++  episode: 278  - step: 2  ++++++++++++

					      +++++ Training +++++
1/1 - 0s - loss: 2.2445 - _timestamp: 1653930842.0000 - _runtime: 370.0000 - 11ms/epoch - 11ms/step

				++++++++++++  episode: 278  - step: 3  +++++++

In [ ]:
wandb.finish()

Energy,█▄▂▄█▆▆▃█▆▅▂▂▃▅▄▁▅▅▄▆█▆▄▆██▇▇▇██▇▆█▄▅▆█▄
Episode,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
Step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,▅▅▆▅▃▅▄▄▃▃▄▄▄█▃▃▃▃▃▄▂▂▃▄▂▇▂▁▇▆▃▂▄▂▄▃▁▂▂▂
Energy,6.68031
Episode,299
Step,2699
epoch,0
loss,2.87667


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9515997f-d5de-4c93-8ae3-fa9b47581edf' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>